In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import lightgbm as lgb
import torch
import random
import re
from collections import defaultdict

import utils

In [2]:
class CFG:
    make_folder = True
    note_num = "StackingOpt/EDA2001"
    seed = 42


OUTPUT_DIR = f'H:/study/output/{CFG.note_num}/'

if CFG.make_folder:
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)


# 各IDの実測値のデータ数
- 201308150000_201408010000の7時から18時（予測時間)までの欠損率等を計算する
- データの重複を消して、6:30から18:00までをsaveする(30分前の発電量で6:30を使う)。

In [27]:
# folder_path = r"H:\study\preprocessing_data\1_twoweeks_nv"に保存されている全てのid
folder_path = r"H:\study\preprocessing_data\1_twoweeks_nv"
file_paths = utils.get_file_paths(folder_path)
id_list = [int(path.split("\\")[-1].split(".")[0]) for path in file_paths]

# 緯度、経度等が分かるid
id_all_data = pd.read_csv("H:\study\output\StackingOpt\EDA006\id_all_data.csv")
id_all_list = id_all_data.id.tolist()

#　共通するid
common_id_list = list(set(id_list)&set(id_all_list))
common_id_list = [str(num).zfill(10) for num in common_id_list]

In [4]:
#検証に使うデータの期間
start_date = "201308150000"
end_date = "201408010000"
date_list = utils.create_time_series_data2(start_date,end_date)
date_list = [datetime.strptime(ts, '%Y%m%d%H%M') for ts in date_list]

In [132]:
OUTPUT_DIR

'H:/study/output/StackingOpt/EDA2001/'

In [5]:
df_list = []
month_count = []
id_list = []
for i,id in enumerate(common_id_list):
    df = utils.get_preprocessing_data2(id)
    df = df[df.datetime.isin(date_list)]
    df.drop_duplicates(subset=["datetime"],inplace=True)
    df.reset_index(drop=True)
    df[["datetime","flag","nv","two_weeks_max","id","generation"]].to_csv(OUTPUT_DIR + f"{id}_201308150000_201408010000.csv",index=False)

    df_list.append(df)
    month_count.append(df[~((df.hour==6)&(df.minute==30))].groupby("month")["generation"].count())
    id_list.append(int(id))
    
    if i%(0.05*int(len(common_id_list)))==0:
        print(f"{i/len(common_id_list)*100}_finish")
    # if i >= 10:
    #     break

df_month = pd.DataFrame(month_count)
df_month["id"] = id_list


date_list2 = [dt for dt in date_list if not ((dt.hour==6)&(dt.minute==30))]

month_count = defaultdict(int)
for dt in date_list2:
    month_count[dt.month] += 1

for month in range(1,13):
    df_month[f"{month}_%"] = df_month[month]/month_count[month] 

df_month.to_csv(OUTPUT_DIR + "df_month.csv",index=False)

0.0_finish


In [7]:
s = pd.read_csv(OUTPUT_DIR + "df_month.csv")

# オプティカルフローのidごとのデータ数

In [25]:
start_date = "201308150000"
end_date = "201408010000"
date_list = utils.create_time_series_data(start_date,end_date)

save_file_list = []
for date in date_list:
    save_file = pd.read_csv('H:/study/output/StackingOpt/EDA006/'+f"pred_id_all_data_{date}.csv")
    save_file_list.append(save_file)
    
save_files = pd.concat(save_file_list)

In [29]:
save_files["datetime"] = pd.to_datetime(save_files["datetime"])
save_files["month"] = save_files["datetime"].dt.month

In [56]:
pred_count = []
pred_interpolated_count = []
id_list = []
for id,group in save_files.groupby("id"):
    pred_count.append(group.groupby("month")["pred"].count())
    pred_interpolated_count.append(group.groupby("month")["pred_interpolated"].count())
    id_list.append(id)
    #break

In [60]:
df_pred = pd.DataFrame(pred_count)
df_pred_interpolated = pd.DataFrame(pred_interpolated_count)

df_pred["id"] = id_list
df_pred_interpolated["id"] = id_list

In [62]:
#date_list = [datetime.strptime(ts, '%Y%m%d%H%M') for ts in date_list]
date_list2 = [dt for dt in date_list if not ((dt.hour==6)&(dt.minute==30))]

month_count = defaultdict(int)
for dt in date_list2:
    month_count[dt.month] += 1

for month in range(1,13):
    df_pred[f"{month}_%"] = df_pred[month]/month_count[month] 
    df_pred_interpolated[f"{month}_%"] = df_pred_interpolated[month]/month_count[month] 

In [66]:
df_pred.to_csv(OUTPUT_DIR + "df_pred.csv",index=False)
df_pred_interpolated.to_csv(OUTPUT_DIR + "df_pred_interpolated.csv",index=False)

# 実測値とオプティカルフローのデータ数

-　期間内で実測値、予測値共に欠損のないPVは397台、これを評価対象のPVとしていきたい。

In [125]:
# folder_path = r"H:\study\preprocessing_data\1_twoweeks_nv"に保存されている全てのid
folder_path = r"H:\study\preprocessing_data\1_twoweeks_nv"
file_paths = utils.get_file_paths(folder_path)
id_list = [int(path.split("\\")[-1].split(".")[0]) for path in file_paths]

# 緯度、経度等が分かるid
id_all_data = pd.read_csv("H:\study\output\StackingOpt\EDA006\id_all_data.csv")
id_all_list = id_all_data.id.tolist()

#　共通するid
common_id_list = list(set(id_list)&set(id_all_list))


#検証に使うデータの期間
start_date = "201308150000"
end_date = "201408010000"
date_list = utils.create_time_series_data2(start_date,end_date)
date_list = [datetime.strptime(ts, '%Y%m%d%H%M') for ts in date_list]
date_list2 = [dt for dt in date_list if not ((dt.hour==6)&(dt.minute==30))]

# 前のセルでsaveしたデータ
df_month = pd.read_csv(OUTPUT_DIR + "df_month.csv")
df_pred = pd.read_csv(OUTPUT_DIR + "df_pred.csv")
df_pred_interpolated = pd.read_csv(OUTPUT_DIR + "df_pred_interpolated.csv")

df_pred = df_pred[df_pred.id.isin(common_id_list)]
df_pred_interpolated = df_pred_interpolated[df_pred_interpolated.id.isin(common_id_list)]

df_month["all"] = df_month[[str(month) for month in range(1,13)]].sum(axis=1)
df_pred_interpolated["all"] = df_pred_interpolated[[str(month) for month in range(1,13)]].sum(axis=1)
df_pred["all"] = df_pred[[str(month) for month in range(1,13)]].sum(axis=1)

df_month["all_%"] = df_month["all"]/len(date_list2)
df_pred_interpolated["all_%"] = df_pred_interpolated["all"]/len(date_list2)
df_pred["all_%"] = df_pred["all"]/len(date_list2)

In [128]:
id_month = df_month[df_month["all_%"]==1].id.tolist()
id_pred_interpolated = df_pred_interpolated[df_pred_interpolated["all_%"]==1].id.tolist()
ids = list(set(id_pred_interpolated)&set(id_month))
print(len(ids))

397


In [131]:
print(ids)

[6240000001, 6240000006, 2720000011, 2720000013, 2450000001, 2130000002, 1010000002, 2130000001, 2450000002, 2450000003, 1810000005, 6130000007, 1010000029, 1010000032, 1010000048, 1010000058, 1010000088, 1010000098, 1010000105, 1010000110, 2340000001, 6180000001, 6180000002, 6180000003, 1010000137, 6500000013, 6500000014, 6500000015, 6340000019, 1010000190, 2070000001, 6710000001, 2070000003, 6710000002, 6710000004, 2230000001, 2230000002, 2230000003, 6550000001, 6550000002, 2390000002, 2390000003, 2390000004, 2390000005, 1270000014, 1270000015, 1270000016, 1750000010, 1270000038, 6070000038, 1010000297, 1010000298, 1010000324, 6070000074, 6070000096, 1010000353, 6070000097, 2110000003, 6760000001, 5000000001, 2760000005, 2440000014, 1160000020, 5000000023, 1160000033, 1160000040, 564, 1160000059, 1160000101, 5000000105, 1160000113, 5000000128, 5000000129, 5000000130, 6010000002, 6010000003, 1210000005, 1210000006, 1210000009, 1210000010, 6650000011, 1210000018, 1690000027, 1160000156

In [124]:
import folium

data = id_all_data[id_all_data.id.isin(ids)]
# 地図の中心座標を設定 (ここでは最初の緯度・経度)
map_center = data.iloc[0][['id_lat', 'id_lng']].tolist()

# 地図を生成
m = folium.Map(location=map_center, zoom_start=6)

# 各地点にマーカーを追加
for _, row in data.iterrows():
    folium.Marker([row['id_lat'], row['id_lng']]).add_to(m)

m

In [122]:
id_month = df_month[df_month["all_%"]==1].id.tolist()
id_pred = df_pred[df_pred["all_%"]==1].id.tolist()
ids2 = list(set(id_pred)&set(id_month))
print(len(ids2))


data = id_all_data[id_all_data.id.isin(ids2)]
# 地図の中心座標を設定 (ここでは最初の緯度・経度)
map_center = data.iloc[0][['id_lat', 'id_lng']].tolist()

# 地図を生成
m = folium.Map(location=map_center, zoom_start=6)

# 各地点にマーカーを追加
for _, row in data.iterrows():
    folium.Marker([row['id_lat'], row['id_lng']]).add_to(m)

m

40


# 6:30も含めた欠損率(30分前の発電量を特徴量に使う為)
- 7:00~18:00で実測値に欠損のない397台のPVに限定して調べる。→390台が残った。

In [3]:
# folder_path = r"H:\study\preprocessing_data\1_twoweeks_nv"に保存されている全てのid
folder_path = r"H:\study\preprocessing_data\1_twoweeks_nv"
file_paths = utils.get_file_paths(folder_path)
id_list = [int(path.split("\\")[-1].split(".")[0]) for path in file_paths]

# 緯度、経度等が分かるid
id_all_data = pd.read_csv("H:\study\output\StackingOpt\EDA006\id_all_data.csv")
id_all_list = id_all_data.id.tolist()

#　共通するid
common_id_list = list(set(id_list)&set(id_all_list))


#検証に使うデータの期間
start_date = "201308150000"
end_date = "201408010000"
date_list = utils.create_time_series_data2(start_date,end_date)
date_list = [datetime.strptime(ts, '%Y%m%d%H%M') for ts in date_list]
date_list2 = [dt for dt in date_list if not ((dt.hour==6)&(dt.minute==30))]

# 前のセルでsaveしたデータ
df_month = pd.read_csv(OUTPUT_DIR + "df_month.csv")
df_pred = pd.read_csv(OUTPUT_DIR + "df_pred.csv")
df_pred_interpolated = pd.read_csv(OUTPUT_DIR + "df_pred_interpolated.csv")

df_pred = df_pred[df_pred.id.isin(common_id_list)]
df_pred_interpolated = df_pred_interpolated[df_pred_interpolated.id.isin(common_id_list)]

df_month["all"] = df_month[[str(month) for month in range(1,13)]].sum(axis=1)
df_pred_interpolated["all"] = df_pred_interpolated[[str(month) for month in range(1,13)]].sum(axis=1)
df_pred["all"] = df_pred[[str(month) for month in range(1,13)]].sum(axis=1)

df_month["all_%"] = df_month["all"]/len(date_list2)
df_pred_interpolated["all_%"] = df_pred_interpolated["all"]/len(date_list2)
df_pred["all_%"] = df_pred["all"]/len(date_list2)

In [4]:
id_month = df_month[df_month["all_%"]==1].id.tolist()
id_pred_interpolated = df_pred_interpolated[df_pred_interpolated["all_%"]==1].id.tolist()
ids = list(set(id_pred_interpolated)&set(id_month))
print(len(ids))

397


In [25]:
to_unique_id = [str(num).zfill(10) for num in ids]
df = utils.get_preprocessing_data3(to_unique_id)
print("期間内のデータ数",len(date_list)) 
ids2 = list(df.id.value_counts()[df.id.value_counts()/len(date_list)==1].index)
print("該当するidの数",len(ids2))

期間内のデータ数 8424
該当するidの数 390


In [24]:
print(ids2)

[6240000001, 1670000003, 6310000002, 6310000001, 6900000114, 6900000110, 6900000109, 6900000108, 6900000105, 6900000104, 6900000094, 6900000093, 6900000090, 2100000086, 2100000085, 2100000084, 6900000081, 6900000080, 2100000068, 2100000067, 2100000066, 6910000449, 6910000448, 6900000131, 6950000001, 6900000173, 6900000134, 6900000171, 6900000168, 6900000166, 6900000165, 6900000162, 6900000161, 6900000158, 6910000541, 6900000156, 6900000155, 6900000154, 6900000153, 6900000152, 6900000150, 6900000149, 6900000145, 6900000144, 6900000143, 6900000142, 6630000003, 6630000002, 6910000438, 6900000054, 6900000049, 6900000047, 6910000327, 6910000308, 6910000306, 6910000303, 6910000301, 6910000299, 6910000298, 6910000294, 6910000293, 6910000292, 6910000291, 6910000287, 6910000283, 6910000282, 6910000281, 6910000279, 6910000276, 1730000020, 6910000274, 6910000272, 6910000271, 6910000343, 2420000001, 6900000002, 2070000002, 6900000045, 6900000044, 6900000041, 6900000040, 6900000035, 6900000034, 690